# EDA

In [22]:
!pip install plotly
!pip install boto3==1.19.12
!pip install s3fs

#### Imports

In [23]:
# General
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import os
import numpy as np
import xlsxwriter
import datetime
import boto3


# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV

# Plots
import matplotlib.pyplot as plt
import seaborn as sns

#Warnings
import warnings
warnings.filterwarnings("ignore")

#### Paths

In [24]:
nps_path = 's3://iberia-data-lake/customer/nps_surveys/export_historic/insert_date_ci=2023-08-07/'
iberia_kpis_path = 's3://iberia-data-lake/customer/one_shot/iberia_kpis_monthly/'
dict_path= 's3://iberia-data-lake/customer/nps_surveys/nps_dictionaries'

#### Read data

In [25]:
df_touchpoints = pd.read_csv(f'{dict_path}/nps_dictionary_touchpoints.csv')
df_issues = pd.read_csv(f'{dict_path}/nps_dictionary_issues.csv')

In [26]:
df_touchpoints_issues = df_touchpoints.merge(df_issues[["associated_touchpoint", "issue_type_2"]], left_on = "survey_maritz_name", right_on = "associated_touchpoint", how = "left")
df_touchpoints_issues["issue_type_2"] = df_touchpoints_issues["issue_type_2"].str.lower()
df_touchpoints_issues = df_touchpoints_issues.loc[(df_touchpoints_issues["issue_type_2"].notnull())
                    & (df_touchpoints_issues["issue_type_2"] != "issuewifi")].drop(["survey_maritz_name"], axis = 1)
issues_list = [i for i in df_touchpoints_issues["issue_type_2"].unique()]

In [27]:
import boto3
import pandas as pd

def read_csv_files_from_s3(s3_path):
    # Initialize Boto3 client for S3
    s3_client = boto3.client('s3')

    # Extract bucket name and prefix from the S3 path
    bucket, prefix = s3_path.replace('s3://', '').split('/', 1)

    # Get a list of all keys (object paths) under the specified S3 path
    s3_resource = boto3.resource('s3')
    s3_keys = [item.key for item in s3_resource.Bucket(bucket).objects.filter(Prefix=prefix)]

    # Generate a list of full S3 paths for all CSV files
    preprocess_paths = [f"s3://{bucket}/{key}" for key in s3_keys]

    # Read all CSV files into a list of DataFrames
    dfs = [pd.read_csv(s3_client.get_object(Bucket=bucket, Key=key)['Body']) for key in s3_keys]

    # Concatenate DataFrames into a single DataFrame
    df_concatenated = pd.concat(dfs, axis=0, ignore_index=True)

    return df_concatenated


In [ ]:
# Readd files from paths
df_nps = read_csv_files_from_s3(nps_path)


In [ ]:
df_kpis = read_csv_files_from_s3(iberia_kpis_path)

In [ ]:
df_nps['date_flight_local'].max()

In [ ]:
df_kpis

#### NPS main information analysis

Filter out null tickets

In [ ]:
condition_1 = (df_nps['operating_airline_code'].isin(['IB', 'YW']))
condition_2 = ((df_nps['invitegroup_ib'] != 3) | (df_nps['invitegroup_ib'].isnull()))
condition_3 = (df_nps['invitegroup'] == 2)

df_nps_tkt = df_nps.loc[condition_1 & (condition_2 & condition_3)]


In [ ]:
#df_nps_tkt_filtered = df_nps_tkt.dropna(subset=['ticket_num'])

In [ ]:
df_nps_tkt.shape


#### Cast and format date columns

In [ ]:
datetime_features = ['date_flight_local', 'scheduled_departure_time_local', 'scheduled_arrival_time_local', 'real_departure_time_local',
                     'real_arrival_time_local', 'started']
columns_to_cross_kpis=['cabin_in_surveyed_flight','haul']
columns_ext = ['tier_level', 'language_code', 'seat_no', 'volume_of_bags', 'number_of_child_in_the_booking', 'number_of_infant_in_the_booking',
              'number_of_people_in_the_booking', 'country_code', 'customer_journey_origin', 'customer_journey_destination', 'number_of_flights_in_journey',
              'order_of_flight_in_journey', 'marketing_airline_code', 'overall_haul', 'weight_category', 'ff_number', 'ticket_num', 'operating_airline_code',
               'nps_category', 'nps_100', 'group_age_survey', 'gender', 'promoter_binary', 'detractor_binary'] # invite_group
touchpoints = ['bkg_100_booking', 'bkg_200_journey_preparation', 'pfl_100_checkin', 'pfl_200_security', 'pfl_300_lounge',
               'pfl_500_boarding', 'ifl_300_cabin', 'ifl_200_flight_crew_annoucements', 'ifl_600_wifi', 'ifl_500_ife',
               'ifl_400_food_drink', 'ifl_100_cabin_crew', 'arr_100_arrivals', 'con_100_connections', 'pun_100_punctuality',
               'loy_200_loyalty_programme', 'inm_400_issues_response', 'img_310_ease_contact_phone']
survey_fields = ['cla_600_wifi_t_f', 'tvl_journey_reason']

for feat in datetime_features:
    if feat in ['scheduled_departure_time_local', 'scheduled_arrival_time_local', 'real_departure_time_local', 'real_arrival_time_local','date_flight_local']:
        df_nps_tkt[feat] = pd.to_datetime(df_nps_tkt[feat], format="%Y%m%d %H:%M:%S", errors = 'coerce')
    else:
        df_nps_tkt[feat] = pd.to_datetime(df_nps_tkt[feat], errors = 'ignore')
df_nps_tkt['time_spent_hrminsec'] = pd.to_timedelta(df_nps_tkt['time_spent_hrminsec']).dt.total_seconds()
df_nps_tkt['started_hour'] = df_nps_tkt['started'].dt.hour
df_nps_tkt['year_flight'] = df_nps_tkt['date_flight_local'].dt.year
df_nps_tkt['month_flight'] = df_nps_tkt['date_flight_local'].dt.month
df_nps_tkt['day_flight'] = df_nps_tkt['date_flight_local'].dt.day
df_nps_tkt['weekday_flight'] = df_nps_tkt['date_flight_local'].dt.weekday
df_nps_tkt['is_weekend_or_friday_flight'] = df_nps_tkt['weekday_flight'].apply(lambda x: 1 if x in [5, 6,7] else 0)
df_nps_tkt['delay_departure'] = (df_nps_tkt['real_departure_time_local'] - df_nps_tkt['scheduled_departure_time_local']).dt.total_seconds()/60
df_nps_tkt['delay_arrival'] = (df_nps_tkt['real_arrival_time_local'] - df_nps_tkt['scheduled_arrival_time_local']).dt.total_seconds()/60
datetime_features = datetime_features + ['time_spent_hrminsec', 'started_hour', 'year_flight', 'month_flight',
                                         'day_flight', 'weekday_flight', 'is_weekend_or_friday_flight']

In [ ]:
df_nps_tkt['date_flight_local'].max()

#### Create flag promoter and detractor

In [ ]:
df_nps_tkt["promoter_binary"] = df_nps_tkt["nps_100"].apply(lambda x: 1 if x == "Promoter" else 0)
df_nps_tkt["detractor_binary"] = df_nps_tkt["nps_100"].apply(lambda x: 1 if x == "Detractor" else 0)

#### Create some features

In [ ]:
def wifi_var(df):

    df["wifi_not_working"] = df["cla_600_wifi_t_f"].apply(lambda x: 1 if x in ["Could not get it to work", "No - I could not get it to work"] else 0)
    df["wifi_used_success"] = df["cla_600_wifi_t_f"].apply(lambda x: 1 if x in ["Yes", "Yes, but not enough"] else 0)
    
    return df

def group_journey_reason(df):
    
    df["tvl_journey_reason"] = df["tvl_journey_reason"].apply(lambda x: 1 if x in ["Business", "Business/work"] else 0)
    
    return df

In [ ]:
df_nps_tkt = wifi_var(df_nps_tkt)
df_nps_tkt = group_journey_reason(df_nps_tkt)

#### Select column

In [ ]:
columns_to_select = datetime_features + columns_ext + touchpoints + ['delay_arrival', 'delay_departure', 'ticket_price']+columns_to_cross_kpis+['monthly_weight']

In [ ]:
df_nps_tkt = df_nps_tkt[columns_to_select]

In [ ]:
df_nps_tkt.head()

In [ ]:
df_nps_tkt['haul'].unique()

In [ ]:
first_date_flight_local = df_nps_tkt['date_flight_local'].min()
print(f"The first date in 'date_flight_local' variable is: {first_date_flight_local}")

Lets check to maximum time spend on survey per nps score.

In [ ]:
df_nps_tkt.groupby('nps_100')['time_spent_hrminsec'].max().plot()
plt.title('Time spent in survey')
plt.show()

Now, lets check how the touchpoints scores correlated with the NPS on the client level. Later we will check the same thing with the aggregated features.

In [ ]:
correlation_fields = ['bkg_100_booking', 'bkg_200_journey_preparation', 'pfl_100_checkin', 'pfl_200_security', 'pfl_300_lounge',
      'pfl_500_boarding', 'ifl_300_cabin', 'ifl_200_flight_crew_annoucements', 'ifl_600_wifi', 'ifl_500_ife',
      'ifl_400_food_drink', 'ifl_100_cabin_crew', 'arr_100_arrivals', 'con_100_connections', 'pun_100_punctuality',
      'loy_200_loyalty_programme', 'inm_400_issues_response', 'img_310_ease_contact_phone', 'nps_100']

In [ ]:
df_nps_corr=df_nps_tkt[correlation_fields]

In [ ]:
df_nps_corr.corr()

#### Agregate on week, calculate NPS, NPS_adjusted, stats of the touchpoints and stats of the touchpoints adjusted, and check time dependencies:

##### HELPER FUNCTIONS

In [ ]:
def calculate_nps(promoters, detractors, passives):
    total_responses = promoters + detractors + passives
    nps = 100 * (promoters - detractors) / total_responses
    return nps

def calculate_aggregated_features(df, variables, time_frequency='M', calculate_satisfaction=True):
    touchpoints = [var for var in variables if calculate_satisfaction]
    non_touchpoints = [var for var in variables if not calculate_satisfaction]

    # Step 1: Resample data to the specified time frequency and calculate NPS for each time period
    resampled_data = df.resample(time_frequency, on='date_flight_local').agg({
        'nps_100': lambda x: calculate_nps(sum((x == 9) | (x == 10)), sum(x <= 6), sum((x == 7) | (x == 8)))
    })

    # Step 2: Calculate the aggregated features for each categorical variable
    aggregated_features_data = pd.DataFrame({
        'feature_date': resampled_data.index,
        'NPS': resampled_data['nps_100']
    })

    for col in touchpoints:
        aggregated_features_data[f'{col}_satisfaction'] = df.resample(time_frequency, on='date_flight_local')[col].apply(lambda x: x[x >= 8].count() / x.count())
        aggregated_features_data[f'{col}_sum'] = df.resample(time_frequency, on='date_flight_local')[col].sum()
        aggregated_features_data[f'{col}_std'] = df.resample(time_frequency, on='date_flight_local')[col].std()
        aggregated_features_data[f'{col}_mean'] = df.resample(time_frequency, on='date_flight_local')[col].mean()
        aggregated_features_data[f'{col}_not_nulls'] = df.resample(time_frequency, on='date_flight_local')[col].apply(lambda x: x.notnull().sum())

    for col in non_touchpoints:
        aggregated_features_data[f'{col}_sum'] = df.resample(time_frequency, on='date_flight_local')[col].sum()
        aggregated_features_data[f'{col}_std'] = df.resample(time_frequency, on='date_flight_local')[col].std()
        aggregated_features_data[f'{col}_mean'] = df.resample(time_frequency, on='date_flight_local')[col].mean()
        aggregated_features_data[f'{col}_not_nulls'] = df.resample(time_frequency, on='date_flight_local')[col].apply(lambda x: x.notnull().sum())
        aggregated_features_data[f'{col}_max'] = df.resample(time_frequency, on='date_flight_local')[col].max()  # Add max calculation
        aggregated_features_data[f'{col}_min'] = df.resample(time_frequency, on='date_flight_local')[col].min()  # Add min calculation

    return aggregated_features_data



In [ ]:
def get_top_correlated_features(df, string_ending, n):
    # Calculate the correlation matrix
    correlation_matrix = df.corr()

    # Get the correlation values of each feature with NPS and sort them in descending order
    nps_correlations = correlation_matrix['NPS'].drop('NPS').sort_values(ascending=False)

    # Filter the correlations for features that end with the specified "_string"
    filtered_correlations = nps_correlations[nps_correlations.index.str.endswith(string_ending)]

    # Get the top 'n' features with the highest correlations among "_string" features
    top_n_correlated_features = filtered_correlations.nlargest(n).index.tolist()

    return top_n_correlated_features


In [ ]:
def plot_time_series_by_years(dataframe, numerical_vars_list, years):
    df=dataframe.copy()
    
    num_rows = len(numerical_vars_list)
    num_years = len(years)

    # Normalize the selected numeric variables and 'NPS' column
    for var in df.columns:
        if var.startswith('NPS') or var in numerical_vars_list[0]:
            df[var] = (df[var] - df[var].min()) / (df[var].max() - df[var].min())

    # Melt the DataFrame to combine numerical variable columns into a single column
    melted_data = pd.melt(df, id_vars=['feature_date'], value_vars=numerical_vars_list[0] + ['NPS'],
                          var_name='variable', value_name='value')

    # Filter data for the specified years
    melted_data_years = melted_data[melted_data['feature_date'].dt.year.isin(years)]
    
    # Create subplots for each row and each year
    fig, axes = plt.subplots(num_rows, num_years, figsize=(20 * num_years, 12 * num_rows), sharex='col', sharey='row')

    for row_idx, numerical_vars in enumerate(numerical_vars_list):
        for col_idx, year in enumerate(years):
            # Plot for the current row and year
            sns.lineplot(x='feature_date', y='value', hue='variable', data=melted_data_years[melted_data_years['feature_date'].dt.year == year], ax=axes[row_idx, col_idx])
            axes[row_idx, col_idx].set_title(f'Normalized Time Series for NPS and Numerical Variables (Year {year})')
            axes[row_idx, col_idx].set_xlabel('Date')
            axes[row_idx, col_idx].set_ylabel('Normalized Values')
            axes[row_idx, col_idx].tick_params(axis='x', rotation=45)  # Rotate x-axis labels for better readability
            axes[row_idx, col_idx].xaxis.set_major_locator(plt.MaxNLocator(nbins=10))  # Set the number of x-axis tick marks

    plt.tight_layout()
    plt.show()

In [ ]:
from calendar import month_name as mn

def plot_nps_time_series(df, years):
    # Filter data for the specified years
    melted_data_years = df[df['feature_date'].dt.year.isin(years)]

    # Create a new 'month' column to represent the month of each feature_date
    melted_data_years['month'] = melted_data_years['feature_date'].dt.month

    # Convert the 'month' column to categorical and ordered
    months = mn[1:]
    melted_data_years['month'] = pd.Categorical(melted_data_years['month'], categories=range(1, 13), ordered=True)

    fig, ax = plt.subplots(figsize=(10, 6))

    for year in years:
        # Filter data for the current year and 'NPS' column
        data_to_plot = melted_data_years[melted_data_years['feature_date'].dt.year == year]
        # Convert the 'month' and 'NPS' columns to regular arrays
        months_arr = data_to_plot['month'].values
        nps_arr = data_to_plot['NPS'].values
        # Plot the line for the current year and 'NPS'
        ax.plot(months_arr, nps_arr, marker='o', label=f'Year {year}')

    ax.set_title('NPS Time Series')
    ax.set_xlabel('Month')
    ax.set_ylabel('NPS Values')
    ax.set_xticks(range(1, 13))
    ax.set_xticklabels(months, rotation=45)
    ax.legend()

    plt.tight_layout()
    plt.show()





In [ ]:
def plot_time_series_by_vars(dataframe, numerical_vars_list, years):
    df=dataframe.copy()
    # Normalize the selected numeric variables and 'NPS' column
    for var in df.columns:
        if var.startswith('NPS') or var in numerical_vars_list[0]:
            df[var] = (df[var] - df[var].min()) / (df[var].max() - df[var].min())

    # Melt the DataFrame to combine numerical variable columns into a single column
    melted_data = pd.melt(df, id_vars=['feature_date'], value_vars=numerical_vars_list[0] + ['NPS'],
                          var_name='variable', value_name='value')

    # Filter data for the specified years
    melted_data_years = melted_data[melted_data['feature_date'].dt.year.isin(years)]

    # Create a new 'month' column to represent the month of each feature_date
    melted_data_years['month'] = melted_data_years['feature_date'].dt.month

    # Convert the 'month' column to categorical and ordered
    months = mn[1:]
    melted_data_years['month'] = pd.Categorical(melted_data_years['month'], categories=range(1, 13), ordered=True)

    num_groups = len(numerical_vars_list)
    
    for group_idx, numerical_vars in enumerate(numerical_vars_list):
        num_vars = len(numerical_vars)
        fig, axes = plt.subplots(1, num_vars, figsize=(5 * num_vars, 5), sharex='col', sharey='row')

        for var_idx, var in enumerate(numerical_vars):
            for year in years:
                # Filter data for the current year and variable
                data_to_plot = melted_data_years[
                    (melted_data_years['feature_date'].dt.year == year) &
                    (melted_data_years['variable'] == var)
                ]
                # Convert the 'month' and 'value' columns to regular arrays
                months_arr = data_to_plot['month'].values
                values_arr = data_to_plot['value'].values
                # Plot the line for the current year and variable
                axes[var_idx].plot(months_arr, values_arr, marker='o', label=f'Year {year}')
                axes[var_idx].set_title(f'{var}')
                axes[var_idx].set_xlabel('Month')
                axes[var_idx].set_ylabel('Normalized Values')
                axes[var_idx].set_xticks(range(1, 13))
                axes[var_idx].set_xticklabels(months, rotation=45)
                axes[var_idx].legend()

        plt.tight_layout()
        plt.show()

In [ ]:
from scipy.stats import pearsonr, spearmanr

def plot_nps_time_series_with_diff(df, years):

    # Melt the DataFrame to combine numerical variable columns into a single column
    melted_data = pd.melt(df, id_vars=['feature_date'], value_vars=['NPS'],
                          var_name='variable', value_name='value')

    # Filter data for the specified years
    melted_data_years = melted_data[melted_data['feature_date'].dt.year.isin(years)]

    # Create a new 'month' column to represent the month of each feature_date
    melted_data_years['month'] = melted_data_years['feature_date'].dt.month

    # Convert the 'month' column to categorical and ordered
    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    melted_data_years['month'] = pd.Categorical(melted_data_years['month'], categories=range(1, 13), ordered=True)

    fig, ax = plt.subplots(figsize=(8, 4))

    for year in years:
        # Filter data for the current year and 'NPS' column
        data_to_plot = melted_data_years[melted_data_years['feature_date'].dt.year == year]
        # Convert the 'month' and 'NPS' columns to regular arrays
        months_arr = data_to_plot['month'].values
        nps_arr = data_to_plot['value'].values
        # Plot the line for the current year and 'NPS' values
        ax.plot(months_arr, nps_arr, marker='o', label=f'NPS (Year {year})')

        # Calculate the differences between consecutive NPS values
        nps_diff = nps_arr[1:] - nps_arr[:-1]
        # Plot the line for the differences
        ax.plot(months_arr[1:], nps_diff, marker='o', linestyle='dashed', label=f'Differences (Year {year})')

    ax.set_title('NPS Time Series with Differences')
    ax.set_xlabel('Month')
    ax.set_ylabel('NPS Values / Differences')
    ax.legend()

    # Compute correlations between NPS values of the two years
    nps_values_1 = melted_data_years[melted_data_years['feature_date'].dt.year == years[0]]['value'].values
    nps_values_2 = melted_data_years[melted_data_years['feature_date'].dt.year == years[1]]['value'].values
    pearson_corr_values, _ = pearsonr(nps_values_1, nps_values_2)
    spearman_corr_values, _ = spearmanr(nps_values_1, nps_values_2)

    # Compute correlations between NPS differences of the two years
    nps_diff_1 = nps_values_1[1:] - nps_values_1[:-1]
    nps_diff_2 = nps_values_2[1:] - nps_values_2[:-1]
    pearson_corr_diff, _ = pearsonr(nps_diff_1, nps_diff_2)
    spearman_corr_diff, _ = spearmanr(nps_diff_1, nps_diff_2)

    print(f'Correlations between NPS values for Year {years[0]} and Year {years[1]}:')
    print(f'Pearson correlation: {pearson_corr_values:.2f}')
    print(f'Spearman correlation: {spearman_corr_values:.2f}')

    print(f'Correlations between NPS differences for Year {years[0]} and Year {years[1]}:')
    print(f'Pearson correlation: {pearson_corr_diff:.2f}')
    print(f'Spearman correlation: {spearman_corr_diff:.2f}')

    plt.tight_layout()
    plt.show()


##### PLOT DIFFERENT VARIABLES AND DEPENDENCIES

In [ ]:
# Agregate touchpoints on a monthly level
aggregated_features_data_monthly = calculate_aggregated_features(df_nps_tkt, ['delay_arrival', 'delay_departure', 'ticket_price'], time_frequency='M', calculate_satisfaction=False)

In [ ]:
aggregated_features_data_w = calculate_aggregated_features(df_nps_tkt, touchpoints, time_frequency='W', calculate_satisfaction=True)

In [ ]:
# Agregate touchpoints on a monthly level
aggregated_touchpoints_data_monthly = calculate_aggregated_features(df_nps_tkt, touchpoints, time_frequency='M', calculate_satisfaction=True)

In [ ]:
aggregated_touchpoints_data_monthly.head()

In [ ]:
# Assuming 'aggregated_features_data_monthly' contains the DataFrame
years_to_filter = [2019,2022,2023,2020,2021]
filtered_data = aggregated_features_data_w[aggregated_features_data_w['feature_date'].dt.year.isin(years_to_filter)]
data_length = len(filtered_data)
print(data_length)


In [ ]:
# Top correlated (with NPS) features
top_3_correlated_features = get_top_correlated_features(aggregated_features_data_monthly, "", 3)
print(top_3_correlated_features)

#Top corelated features among the _satisfaction ones
top_3_mean_features = get_top_correlated_features(aggregated_features_data_monthly, "_mean", 3)
print(top_3_mean_features)

# Top correlated features among the _not_nulls ones
top_3_max_features = get_top_correlated_features(aggregated_features_data_monthly, "_max", 3)
print(top_3_max_features)

#Plot timeseries for NPS and the most correlated features in years 2019 and 2022
years_to_plot = [2022, 2021, 2020, 2019]  # Add any arbitrary number of years to plot
numerical_vars_list_to_plot = [top_3_correlated_features, top_3_mean_features, top_3_max_features]  # Add any list of numeric variables to plot
plot_time_series_by_years(aggregated_features_data_monthly, numerical_vars_list_to_plot, years_to_plot)

In [ ]:
# Top correlated (with NPS) features
top_3_correlated_touchpoints = get_top_correlated_features(aggregated_touchpoints_data_monthly, "", 3)
print(top_3_correlated_touchpoints)

#Top corelated features among the _satisfaction ones
top_3_satisfaction_touchpoints = get_top_correlated_features(aggregated_touchpoints_data_monthly, "_satisfaction", 3)
print(top_3_satisfaction_touchpoints)

# Top correlated features among the _not_nulls ones
top_3_not_nulls_touchpoints = get_top_correlated_features(aggregated_touchpoints_data_monthly, "_not_nulls", 3)
print(top_3_not_nulls_touchpoints)

#Plot timeseries for NPS and the most correlated features in years 2019 and 2022
years_to_plot = [2022, 2021, 2020, 2019]  # Add any arbitrary number of years to plot
touchpoints_list_to_plot = [top_3_correlated_touchpoints, top_3_not_nulls_touchpoints, top_3_satisfaction_touchpoints]  # Add any list of numeric variables to plot
plot_time_series_by_years(aggregated_touchpoints_data_monthly, touchpoints_list_to_plot, years_to_plot)


In [ ]:
# Assuming you have the DataFrame aggregated_features_data_monthly and a list of numeric variables top_3_not_nulls_features
years_to_plot = [2022, 2019]  # Add any arbitrary number of years to plot
numeric_vars_to_plot = [top_3_correlated_features,top_3_mean_features,top_3_max_features]  

for numeric_vars_list in numeric_vars_to_plot:
    plot_time_series_by_vars(aggregated_features_data_monthly, [numeric_vars_list], years_to_plot)

In [ ]:
# Assuming you have the DataFrame aggregated_features_data_monthly and a list of numeric variables top_3_not_nulls_features
years_to_plot = [2022, 2021, 2020, 2019]  # Add any arbitrary number of years to plot
touchpoints_to_plot = [top_3_correlated_touchpoints,top_3_satisfaction_touchpoints,top_3_not_nulls_touchpoints]  # Add a list of numeric variables to plot (e.g., [['numeric_var_1', 'numeric_var_2']])

for touchpoints_list in touchpoints_to_plot:
    plot_time_series_by_vars(aggregated_touchpoints_data_monthly, [touchpoints_list], years_to_plot)

In [ ]:
plot_nps_time_series(aggregated_touchpoints_data_monthly, [2019,2022,2023])


In [ ]:
nps_bi = pd.read_excel('data.xlsx')
nps_bi

In [ ]:
from calendar import month_name as mn

def plot_nps_time_series(df, nps_bi, years):
    # Filter data for the specified years
    melted_data_years = df[df['feature_date'].dt.year.isin(years)]

    # Create a new 'month' column to represent the month of each feature_date
    melted_data_years['month'] = melted_data_years['feature_date'].dt.month

    # Convert the 'month' column to categorical and ordered
    months = mn[1:]
    melted_data_years['month'] = pd.Categorical(melted_data_years['month'], categories=range(1, 13), ordered=True)

    fig, ax = plt.subplots(figsize=(10, 6))

    for year in years:
        # Filter data for the current year and 'NPS' column
        data_to_plot = melted_data_years[melted_data_years['feature_date'].dt.year == year]
        # Convert the 'month' and 'NPS' columns to regular arrays
        months_arr = data_to_plot['month'].values
        nps_arr = data_to_plot['NPS'].values
        # Plot the line for the current year and 'NPS'
        ax.plot(months_arr, nps_arr, marker='o', label=f'Year {year}')

        # Plot 'nps_bi' data for the current year
        nps_bi_year = f'NPS {year}'
        if nps_bi_year in nps_bi.columns:
            ax.plot(months_arr, nps_bi[nps_bi_year], marker='x', label=f'Year {year} (Marketing)', linestyle='dashed')



In [ ]:
from calendar import month_name as mn

def plot_nps_time_series(df, nps_bi, years):
    # Filter data for the specified years
    melted_data_years = df[df['feature_date'].dt.year.isin(years)]

    # Create a new 'month' column to represent the month of each feature_date
    melted_data_years['month'] = melted_data_years['feature_date'].dt.month

    # Convert the 'month' column to categorical and ordered
    months = mn[1:]
    melted_data_years['month'] = pd.Categorical(melted_data_years['month'], categories=range(1, 13), ordered=True)

    fig, ax = plt.subplots(figsize=(10, 6))

    for year in years:
        # Filter data for the current year and 'NPS' column
        data_to_plot = melted_data_years[melted_data_years['feature_date'].dt.year == year]
        # Convert the 'month' and 'NPS' columns to regular arrays
        months_arr = data_to_plot['month'].values
        nps_arr = data_to_plot['NPS'].values
        # Plot the line for the current year and 'NPS'
        ax.plot(months_arr, nps_arr, marker='o', label=f'Year {year}')

        # Plot 'nps_bi' data for the current year
        nps_bi_year = f'NPS {year}'
        if nps_bi_year in nps_bi.columns:
            ax.plot(months_arr, nps_bi[nps_bi_year], marker='x', label=f'Year {year} (Marketing)', linestyle='dashed')

    ax.set_title('NPS Time Series')
    ax.set_xlabel('Month')
    ax.set_ylabel('NPS Values')
    ax.set_xticks(range(1, 13))
    ax.set_xticklabels(months, rotation=45)
    ax.legend()

    plt.tight_layout()
    plt.show()

# Assuming 'nps_bi' is your marketing team's DataFrame
# and 'df' is your original DataFrame
#plot_nps_time_series(aggregated_touchpoints_data_monthly, nps_bi, years=[2019, 2022, 2023])


In [ ]:
# Plot the NPS comparison and differences
plot_nps_time_series_with_diff(aggregated_touchpoints_data_monthly, [2019, 2020,2021, 2022])


In [ ]:
plot_nps_time_series_with_diff(aggregated_touchpoints_data_monthly, [2019, 2018,2017])


#### Combine monthly aggregates of NPS with Iberia KPIs

In [ ]:
aggregated_features_data_monthly

In [ ]:
df_kpis['haul'].unique()

In [ ]:
import pandas as pd

# Function to calculate NPS
def calculate_nps(promoters, detractors, passives):
    total_responses = promoters + detractors + passives
    nps = 100 * (promoters - detractors) / total_responses
    return nps

def calculate_aggregated_features_with_cross_kpis(df, variables, columns_to_cross_kpis, time_frequency='M', calculate_satisfaction=True):
    touchpoints = [var for var in variables if calculate_satisfaction]
    non_touchpoints = [var for var in variables if not calculate_satisfaction]

    # Combine the values of columns_to_cross_kpis into a new column 'cross_kpis'
    df['cross_kpis'] = df[columns_to_cross_kpis].apply(lambda x: '_'.join(x.values.astype(str)), axis=1)

    # Group the data by 'cross_kpis' and 'date_flight_local' using pd.Grouper on the entire DataFrame
    grouped_data = df.groupby([pd.Grouper(key='cross_kpis'), pd.Grouper(key='date_flight_local', freq=time_frequency)], group_keys=False)

    # Calculate the aggregated features for each group
    aggregated_features_data = pd.DataFrame()
    aggregated_features_data['NPS'] = grouped_data.apply(lambda x: calculate_nps(
        sum((x['nps_100'] == 9) | (x['nps_100'] == 10)),
        sum(x['nps_100'] <= 6),
        sum((x['nps_100'] == 7) | (x['nps_100'] == 8))
    ))

    for col in touchpoints + non_touchpoints:
        aggregated_features_data[f'{col}_sum'] = grouped_data[col].sum()
        aggregated_features_data[f'{col}_std'] = grouped_data[col].std()
        aggregated_features_data[f'{col}_mean'] = grouped_data[col].mean()
        aggregated_features_data[f'{col}_not_nulls'] = grouped_data[col].apply(lambda x: x.notnull().sum())
        if col in touchpoints:
            aggregated_features_data[f'{col}_satisfaction'] = grouped_data[col].apply(lambda x: x[x >= 8].count() / x.count())
        else:
            aggregated_features_data[f'{col}_max'] = grouped_data[col].max()
            aggregated_features_data[f'{col}_min'] = grouped_data[col].min()
            

    # Reset the index of aggregated_features_data
    aggregated_features_data.reset_index(inplace=True)

    # Split the 'cross_kpis' column back into individual columns and concatenate them to aggregated_features_data
    split_columns = aggregated_features_data['cross_kpis'].str.split('_', expand=True)
    split_columns.columns = columns_to_cross_kpis
    aggregated_features_data = pd.concat([split_columns, aggregated_features_data], axis=1)
    
    # Drop the temporary 'cross_kpis' column
    aggregated_features_data.drop(columns=['cross_kpis'], inplace=True)
    
    return aggregated_features_data



# Assuming you have already defined the required variables 'touchpoints', 'non_touchpoints', 'df_nps_tkt', and 'columns_to_cross_kpis'
result_df = calculate_aggregated_features_with_cross_kpis(df_nps_tkt, touchpoints, columns_to_cross_kpis, time_frequency='M', calculate_satisfaction=True)




In [ ]:
df_kpis

In [ ]:
df_kpis['cabin'].unique()

In [ ]:
df_kpis['haul'].unique()

In [ ]:
len(df_kpis)

In [ ]:
result_df['cabin_in_surveyed_flight'].unique()

In [ ]:
result_df['haul'].unique()

In [ ]:
len(result_df[result_df['date_flight_local'].dt.year >= 2019])

In [ ]:
result_df['cabin_in_surveyed_flight']=result_df['cabin_in_surveyed_flight'].replace('Business','Premium')

In [ ]:
result_df[result_df['date_flight_local'].dt.year >= 2019]

In [ ]:
# Convert the 'date_flight_local' column in result_df to separate 'year' and 'month' columns
result_df['year'] = result_df['date_flight_local'].dt.year
result_df['month'] = result_df['date_flight_local'].dt.month

# Perform the merge based on the specified columns
merged_df = result_df.merge(df_kpis, 
                            left_on=['year', 'month', 'cabin_in_surveyed_flight', 'haul'], 
                            right_on=['year', 'month', 'cabin', 'haul'], 
                            how='inner')

# Drop the redundant columns from the merged dataframe
merged_df.drop(columns=['cabin_in_surveyed_flight'], inplace=True)

# Reorder the columns to place 'year', 'month', 'cabin', and 'haul' at the beginning
cols_to_move = ['date_flight_local','year', 'month', 'cabin', 'haul']
merged_df = merged_df[cols_to_move + [col for col in merged_df.columns if col not in cols_to_move]]


In [ ]:
merged_df.to_csv('merged_df_complete', index=False)

In [ ]:
# Top correlated (with NPS) features
top_5_correlated_touchpoints = get_top_correlated_features(aggregated_touchpoints_data_monthly, "", 5)
print(top_5_correlated_touchpoints)

#Top corelated features among the _satisfaction ones
top_5_satisfaction_touchpoints = get_top_correlated_features(aggregated_touchpoints_data_monthly, "_satisfaction", 5)
print(top_5_satisfaction_touchpoints)

# Top correlated features among the _not_nulls ones
top_5_not_nulls_touchpoints = get_top_correlated_features(aggregated_touchpoints_data_monthly, "_not_nulls", 5)
print(top_5_not_nulls_touchpoints)

#Plot timeseries for NPS and the most correlated features in years 2019 and 2022
years_to_plot = [2022, 2021, 2020, 2019]  # Add any arbitrary number of years to plot
touchpoints_list_to_plot = [top_5_correlated_touchpoints, top_5_not_nulls_touchpoints, top_5_satisfaction_touchpoints]  # Add any list of numeric variables to plot
plot_time_series_by_years(aggregated_touchpoints_data_monthly, touchpoints_list_to_plot, years_to_plot)

In [ ]:
# Define the columns to keep in the desired order
cols_to_keep = ['date_flight_local','year', 'month', 'cabin', 'haul'] + top_5_satisfaction_touchpoints + ['load_factor', 'mean_price', 'deviation_price'] + [col for col in merged_df.columns if col.startswith('otp')]+['NPS']

# Keep only the specified columns in the desired order
merged_df = merged_df[cols_to_keep]

In [ ]:
merged_df.to_csv('merged_df', index=False)

In [ ]:
# Combine 'year' and 'month' columns into a new column 'year_month' in df_kpis
df_kpis['year_month'] = df_kpis['year'].astype(str) + '-' + df_kpis['month'].astype(str)

# Combine 'year' and 'month' columns into a new column 'year_month' in merged_df
merged_df['year_month'] = merged_df['year'].astype(str) + '-' + merged_df['month'].astype(str)

# Check which combinations are missing in merged_df
missing_combinations = df_kpis[~df_kpis['year_month'].isin(merged_df['year_month'])]

# Display the missing combinations
print(missing_combinations)

merged_df.drop(columns=['year_month'], inplace=True)

#### MVP model trainning using Darts

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
merged_df=pd.read_csv('merged_df')

In [ ]:
merged_df.head()


##### Installations

In [ ]:
pip install darts

In [ ]:
pip install lightgbm

In [ ]:
import darts
from darts import TimeSeries
from darts.utils.timeseries_generation import (
    gaussian_timeseries,
    linear_timeseries,
    sine_timeseries,
)

from darts.metrics import mape, smape, mae
from darts.dataprocessing.transformers import Scaler
from darts.utils.timeseries_generation import datetime_attribute_timeseries

from sklearn.linear_model import BayesianRidge
from sklearn.ensemble import RandomForestRegressor

from darts.models import LightGBMModel

import matplotlib.pyplot as plt




import logging

logging.disable(logging.CRITICAL)

import warnings

warnings.filterwarnings("ignore")



##### First approach: 1 dataframe, 5 group cols

In [ ]:
# Get the categorical columns
categorical_cols = ["cabin","haul"]

merged_df['date_flight_local'] = pd.to_datetime(merged_df['date_flight_local'])

# One-hot encode the categorical columns
merged_df_encoded = pd.get_dummies(merged_df, columns=categorical_cols)

merged_df_encoded = merged_df_encoded[merged_df_encoded['date_flight_local'].dt.year.isin([2019, 2022, 2023])]
import pandas as pd

merged_df_encoded['date_flight_local'] = merged_df_encoded['date_flight_local'].apply(lambda x: x.replace(year=2021) if x.year == 2019 else x)

In [ ]:
%%capture
top_5_satisfaction_touchpoints=['pun_100_punctuality_satisfaction', 'ifl_100_cabin_crew_satisfaction', 'pfl_500_boarding_satisfaction', 'ifl_300_cabin_satisfaction', 'bkg_200_journey_preparation_satisfaction']
features=top_5_satisfaction_touchpoints + ['load_factor', 'mean_price', 'deviation_price'] + [col for col in merged_df.columns if col.startswith('otp')]

NPS_ts= TimeSeries.from_group_dataframe(
    merged_df_encoded,
    time_col="date_flight_local",
    group_cols=["cabin_Economy","cabin_Premium","cabin_Premium Economy","haul_SH","haul_LH"],  # individual time series are extracted by grouping `df` by `group_cols`
    value_cols=['NPS'],
    fillna_value=0,
    freq='M'# optionally, specify the time varying columns
)

covariates_ts = TimeSeries.from_group_dataframe(
    merged_df_encoded,
    time_col="date_flight_local",
    group_cols=["cabin_Economy","cabin_Premium","cabin_Premium Economy","haul_SH","haul_LH"],
    value_cols=features,  # Use only the features as covariates
    fillna_value=0,
    freq='M'
)

train_nps = []
val_nps = []
train_covariates = []
val_covariates = []

for ts in NPS_ts:
    train_nps.append(ts[:-12])  # Append the first part of the time series to train_nps
    val_nps.append(ts[-12:])   # Append the last 12 elements (validation data) to val_nps

for covariate_ts in covariates_ts:
    train_covariates.append(covariate_ts[:-12])  # Append the first part of the covariate time series to train_covariates
    val_covariates.append(covariate_ts[-12:])    # Append the last 12 elements (validation data) to val_covariates
    
from darts.models import LightGBMModel

quantiles = [0.25, 0.5, 0.75]

model = LightGBMModel(
    lags=12,
    quantiles=quantiles,
    lags_future_covariates=[-12],
    likelihood="quantile",
    fit_intercept=False,
    output_chunk_lenth=12,
)

model.fit(
    train_nps, future_covariates=train_covariates
)


In [ ]:
print(f"\n{len(NPS_ts)} series were extracted from the input DataFrame")
for i, ts in enumerate(NPS_ts):
    print(f"Series {i}")
    ts["NPS"].plot(label=f"NPS_series_{i}")

In [ ]:
print(f"\n{len(covariates_ts)} series were extracted from the input DataFrame")
for i, ts in enumerate(covariates_ts):
    print(f"Series {i}")
    ts["load_factor"].plot(label=f"NPS_series_{i}")

In [ ]:
%%capture
pred_nps = model.predict(
    series=train_nps,
    future_covariates=train_covariates,
    n=12,
)


In [ ]:
plt.figure(figsize=(12, 15))

for i in range(5):
    plt.subplot(5, 1, i + 1)
    NPS_ts[i].plot(label="Actual NPS")
    pred_nps[i].plot(label="Forecast NPS")
    plt.legend()
    plt.title(f"Plot for Index {i}")

plt.tight_layout()
# Save the plot as a PNG file
plt.savefig("forecast_plots.png")
plt.show()

In [ ]:
mape(NPS_ts[0], pred_nps[0])

In [ ]:
len(train_nps[0])

In [ ]:
%%capture
start_index = (len(train_nps[0]) - 11)/len(train_nps[0])
backtest = model.historical_forecasts(
    train_nps, future_covariates=train_covariates, start=start_index, forecast_horizon=12, verbose=True
)

In [ ]:
for i in range(5):
    mape_value = mape(backtest[i], train_nps[i])
    print("MAPE = %.2f" % mape_value)
    
    plt.figure(figsize=(8,4))
    train_nps[i].plot(label="Train NPS")
    backtest[i].plot(label="Backtest NPS")
    plt.legend()
    plt.title(f"Plot for Index {i}")
    plt.show()


##### Second approach: 5 different dataframes

In [ ]:
merged_df['date_flight_local']=pd.to_datetime(merged_df['date_flight_local'])

merged_df.head()


In [ ]:
grouped_dfs = {}

# Group the original DataFrame by the 'cabin' and 'haul' columns
grouped = merged_df.groupby(['cabin', 'haul'])

# Iterate through each group and create a DataFrame
for group_name, group_data in grouped:
    cabin_value, haul_value = group_name
    group_df = group_data.copy()  # Create a copy of the group's data
    group_df_name = f'{cabin_value}_{haul_value}_df'  # Generate a unique name
    grouped_dfs[group_df_name] = group_df

# Now you have a dictionary containing separate DataFrames for each combination of values
# Access them using the keys in the dictionary
grouped_dfs['Premium Economy_LH_df'] = grouped_dfs['Premium Economy_LH_df'][grouped_dfs['Premium Economy_LH_df']['date_flight_local'].dt.year.isin([2019, 2022, 2023])]

grouped_dfs['Premium Economy_LH_df']['date_flight_local'] = grouped_dfs['Premium Economy_LH_df']['date_flight_local'].apply(lambda x: x.replace(year=2021) if x.year == 2019 else x)

grouped_dfs['Premium Economy_LH_df'] = grouped_dfs['Premium Economy_LH_df'].reset_index()

grouped_dfs['Premium Economy_LH_df']

In [ ]:
import pandas as pd
from darts import TimeSeries
from darts.models import LightGBMModel

# Load the data
df = grouped_dfs['Premium Economy_LH_df']

# Create a new column called "date" that contains the datetime objects
df['date'] = pd.to_datetime(df['date_flight_local'])

df=df.set_index('date')

df=df.fillna(0)


# Create the target time series
nps_ts = TimeSeries.from_series(df['NPS'])

# Create the future covariates time series
top_5_satisfaction_touchpoints=['pun_100_punctuality_satisfaction', 'ifl_100_cabin_crew_satisfaction', 'pfl_500_boarding_satisfaction', 'ifl_300_cabin_satisfaction', 'bkg_200_journey_preparation_satisfaction']
features=top_5_satisfaction_touchpoints + ['load_factor', 'mean_price', 'deviation_price'] + [col for col in merged_df.columns if col.startswith('otp')]
                                   
future_covariates_ts = TimeSeries.from_series(df[features])




In [ ]:
train_ts, val_ts = nps_ts[:-12], nps_ts[-12:]
train_covariates_ts, val_covariates_ts = future_covariates_ts[:-12], future_covariates_ts[-12:]

In [ ]:
# Fit the LightGBM model
model = LightGBMModel(
    lags=12,
    quantiles=[0.25, 0.5, 0.75],
    lags_future_covariates=[-12],  # Use the last 12 elements of covariate for each prediction step
    likelihood="quantile",
    fit_intercept=False,
    output_chunk_length=6
)
model.fit(train_ts, future_covariates=train_covariates_ts)

In [ ]:
# Make predictions on the validation set
pred_nps = model.predict(n=12, series=train_ts, future_covariates=train_covariates_ts)

plt.figure(figsize=(6,4))


nps_ts.plot(label="Actual NPS")
pred_nps.plot(label="Forecast NPS")
plt.legend()
plt.title(f"PE_LH")

plt.tight_layout()
# Save the plot as a PNG file
plt.show()

In [ ]:
top_5_satisfaction_touchpoints=['pun_100_punctuality_satisfaction', 'ifl_100_cabin_crew_satisfaction', 'pfl_500_boarding_satisfaction', 'ifl_300_cabin_satisfaction', 'bkg_200_journey_preparation_satisfaction']
features=top_5_satisfaction_touchpoints + ['load_factor', 'mean_price', 'deviation_price'] + [col for col in merged_df.columns if col.startswith('otp')]

In [ ]:
df.describe()

In [ ]:
futurecovariates_ts 

In [ ]:
train_ts


##### Third approach: old school lagging + regressor

In [ ]:
df=grouped_dfs['Premium Economy_LH_df'].copy()

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import lightgbm as lgb

# List of features to lag
features_to_lag = (
    ['NPS']
)

# Number of lags
lags = 12

# Create lagged features using pandas
for feature in features_to_lag:
    for lag in range(1, lags + 1):
        df[f'lag_{lag}_{feature}'] = df[feature].shift(lag)

# Fill NaN values with the median for each column
df.fillna(df.mean(), inplace=True)

# Split into features and target variable
X = df.drop(columns=['NPS','date_flight_local','cabin','haul'])
y = df['NPS']

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

# Create and train LightGBM model
params = {
    'objective': 'quantile',
    'alpha': 0.25,  # Quantile parameter
    'boosting_type': 'gbdt',
    # Set other hyperparameters
}

model = lgb.LGBMRegressor(**params)
model.fit(X_train, y_train)

# Evaluate the model on validation data
predictions = model.predict(X_val)

df.head()




In [ ]:
from sklearn.metrics import mean_absolute_error

# Assuming you have already trained and evaluated the model
# predictions contains the predicted NPS values on the validation set
# y_val contains the actual NPS values on the validation set

# Compute the MAE
mae = mean_absolute_error(y_val, predictions)

# Print the MAE
print(f"Mean Absolute Error: {mae}")
